# Automatic Deep Research Flow and Observability

Welcome to the second practice lab of this module! 

In this lab, you will continue working on your automatic deep research system. In this lab you will take the crew you built so far, and include it as one of the building blocks of your very first flow.

**What you'll learn:**
- How to build flows using the CrewAI CLI
- Use tracing to achieve monitoring and observability of your crews

## Background

As a research consultant, you're constantly tasked with producing comprehensive reports on diverse topics for demanding clients. You need to build an AI research crew that can rapidly gather, verify, and synthesize information from across the internet, delivering reliable, fact-checked reports that meet tight deadlines and exacting standards regardless of the subject matter.

## General instructions
In this lab you will be presented with a structure of the code, but you will need to complete some of it. 

To successfully run this lab, replace all instances of the placeholder `None` with your own code. Sections where you need to write code will be delimited between `### START CODE HERE ###` and `### END CODE HERE ###`.

**<font color='#5DADEC'>Please make sure to save your work periodically, so you don't lose any progress.</font>**

## Table of contents

- [1. Problem statement](#1)
- [2. Inspecting the flow structure](#2)
- [3. Configure the Flow](#3)
    - [3.1. Define the flow state](#3-1)
    - [3.2. Add persistence to the Flow](#3-2)
    - [3.3. Define the entrypoint](#3-3)
    - [3.4. Decide whether to trigger the Crew or not](#3-4)
    - [3.5. Write the task to answer simple queries](#3-5)
    - [3.6. Write the tasks involved in the research route](#3-6)
        - [3.6.1. Clarification questions](#3-6-1)
        - [3.6.2. Kickoff the crew](#3-6-2)
        - [3.6.3. Summarize the report](#3-6-3)
    - [3.7. Wrap up the flow and give the final answer](#3-7)
    - [3.8 Add monitoring to the FLow](#3-8)
- [4. Plot the Flow graph](#4)
- [5. Run the Flow](#5)

<a id="1"></a>

## 1. Problem statement

In the previous lab you built a deep research crew with the following structure:

<img src="../images/m3-lab1-agents-tasks-diagram.png">

In this lab, you will embed this crew into a Flow, that actually decides whether the deep research crew is needed or not. The flow should look something like this:

<div style="text-align: center;">
<img src='../images/m3-lab2-flow-diagram.png' width=500>
</div>

<a id="2"></a>

## 2. Inspecting the flow structure
In this lab, you will learn how to build a flow, which includes the deep research crew you created in the last lab. 

If you were to create the flow from scratch, you can do so by running the following command on a terminal: 

```bash
crewai create flow deep_research_flow
```

However, just as in the last lab, you are already provided with the structure of the flow, so that we can guide you through the building process. If you inspect the  `deep_research_flow` folder you will find the following structure:
<div style="text-align: center;">
<img src='../images/lab2-folder-structure.png' width=200>
</div>

As you can see, the structure is very similar to the one in the previous lab, only this time inside the `src` folder you get a folder named `crews`. In it you will find all the files you created in the previous lab to create the crew. 

<a id="3"></a>

## 3. Configure the Flow

The crew for this flow is already given to you. The only difference is it is missing the after-kickoff hook, because it will be incorporated into the flow. In this lab you will focus on adding all the missing steps to create the flow. 

Just as you did with the Crew in the previous lab, to build a flow you need to create a Flow object, inside which you will define all the tasks it needs to run, and interconnect them using  decorators. This object is created in the [`src/deep_research_flow/main.py`](deep_research_flow/src/deep_research_flow/main.py) file. This main file looks a bit different from the one for crews, but you will be able to complete it by the end of the lab. 

There are three main decorators you will use to connect the different methods, or tasks, inside the flow:
- `@persist()`: automatically persists the state of the flow.
- `@start()`: indicates the entrypoints of the flow. This are the first methods to run.
- `@listen()`: marks a method as a listener for the output of another task in the Flow. The method decorated with @listen() will be executed when the specified task emits an output.
- `router()`: allows you to define conditional routing logic based on the output of a method.

<a id="3-1"></a>

### 3.1. Define the flow state
The flow state is a group of variables that show the current state of the flow. In this case you will want to define the following values: 
- `user_query`: a string with the query
- `needs_research`: bool indicating whether the research crew needs to be deployed or not
- `research_report`: a string with the report generated by the Crew (if deployed)
- `final_answer`: a string with the final answer

Open the [`src/deep_research_flow/main.py`](deep_research_flow/src/deep_research_flow/main.py) file and fill in the flow state in the `ResearchState` class. For each variable you need to set the type (`int`, `bool`, etc.) and the initial value.

<a id="3-2"></a>

### 3.2. Add persistence to the Flow

You want the state of the flow and all executions to be saved into a database, so you can access them and persist data across different executions. 

1. Add the correct decorator just before defining the `DeepResearchFlow` class.

<a id="3-3"></a>

### 3.3. Define the entrypoint
In this lab, the starting point will ask the user to input the research prompt, and save it in the `user_query` state variable. 

1. Add the correct decorator to the `start_conversation` method inside the `DeepResearchFlow` class. 
2. Take a moment to understand what the code defining the method.

<a id="3-4"></a>

### 3.4. Decide whether to trigger the Crew or not
Next, define the method in charge of deciding if the research should be triggered or not. This task uses a simple LLM call to decide which route in the graph the Flow is going to follow.

1. Set up the decorator (`router`) in the `analyze_query` method
2. Write the missing part in the query for the LLM. Guidelines:
    - If the query requires triggering the research, it should return the word "RESEARCH"
    - If the query does not require the research, and can be handled by a simple LLM, it should return the word "SIMPLE". 
3. Look at the remaining code in this method, pay special attention to the return statements.

<a id="3-5"></a>

### 3.5. Write the task to answer simple queries
If the router task returns "SIMPLE", then you need to run a task that calls a simple LLM to generate the answer to the user's query.

1. Take a look at how the decorator is set up for the `simple_answer` method. Note that it listens for the return value "SIMPLE" of the router task. 
2. Write the query for the LLM. It should give an answer to the user's query.
3. Set up the LLM in charge of giving the answer.
4. Save the response in the state variable.

<a id="3-6"></a>

### 3.6. Write the tasks involved in the research route
If the research is needed the following steps needs to happen
1. Review the query and ask clarification questions if needed
2. Kickoff the crew with the original query plus the additional information. Save the results in the `
3. Save the report to a local final and update the state variable `research_report`
4. Create a summary of the report to use as the final answer. Update the `final_answer` state variable

<a id="3-6-1"></a>

#### 3.6.1. Clarification questions
1. Set up the decorator for the `clarify_query` method. It needs to listen to the the "RESEARCH" output of the router
2. Take some time to understand how the task is implemented.

<a id="3-6-2"></a>

#### 3.6.2. Kickoff the crew

1. Set up the decorator for the `execute_research` method. It should listen to the output of the `clarify_query` task.
2. Kickoff the crew using the `user_query` state variable as input
3. Update the `research_report` state

<a id="3-6-3"></a>

#### 3.6.3. Summarize the report
1. Set up the decorator for the `save_report_and_summarize` method. It should listen to the output of `execute_research`
2. Write the content of the `research_report` state into a markdown file
3. Take a moment to look at the code that calls an LLM to summarize the report

<a id="3-7"></a>

### 3.7. Wrap up the flow and give the final answer
Finally, you are ready to give the final answer to the user. This answer can either be the summary of the report, or the simple answer.

1. Take a look at the decorator for the `return_final_answer`. Since it should execute when either of the branches finishes, you need to use the `_or` conditional logic inside the `@listen` decorator.
2. Look at the rest of the method. It simply prints out the state variables `user_query` and `final answer`.

<a id="3-8"></a>

### 3.8 Add monitoring to the FLow

You are almost done setting up the Flow. As a last step, you want to be able to  monitor your Flow, in order to identify issues early on, and generally keep an eye on the health of the system. That can easily be done by allowing for tracing.
1. Look for the `kickoff()` function inside the `main.py` file you have open. 
2. Create an instance of the Flow. and set the `tracing` argument to `True`. 

Once you run the Flow, it records every step the Flow takes so you can later see exactly how decisions were made and how agents interacted. On execution, you are given a URL on the CrewAI platform where you can explore all this data.

<a id="4"></a>

## 4. Plot the Flow graph

Once you are done with the previous steps, don't forget to save your changes. 
After you are done, you can run the next cell to generate the visualization of the flow you just created. The first time you run this flow, you will see it will take some time because it creates a virtual environment with all the required dependencies, which could take a minute or two.  By the end, it will have created an HTML file with the flow graph inside the [`deep_research_flow`](deep_research_flow) folder.

In [ ]:
# run this cell to avoid warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
! cd deep_research_flow && crewai flow plot


<a id="5"></a>

## 5. Run the Flow
Now you are ready to kickoff the Flow! Since this flow interacts with the user by asking for the question, and possibly some clarifications, you can't run this flow from the Jupyter notebook. You will need to open a terminal and run

```bash
cd M3/ugls/Lab2/deep_research_flow && crewai run
```

You will be prompted for your query. If it is marked as a complex query, you might be asked for clarification. 


**Note**: To open a terminal in a new tab you need to click on `File > New > Terminal`

<div style="text-align: center;">
<img src='../images/open_new_terminal.png' width=550>
</div>

Once it finishes, you should see something like this (might look slightly different if you get the prompt):

```shell
╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ b30ddc1b-43a0-4e36-b9ea-109b8b8d5145. View here:                             │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/b30ddc1b-43a0-4e3 │
│ 6-b9ea-109b8b8d5145?access_code=TRACE-********** , Access Code:              │
│ TRACE-**********                                                             │
╰──────────────────────────────────────────────────────────────────────────────╯
```

Go ahead and access the URL. If the hyperlink is cut, meaning it is not spanning both lines, just copy and paste the full URL in a new browser tab. There you can explore the details of the run, with all the data gathered from the tracing.

Experiment with different queries. Use simple queries like "Hello!" or more complex ones so that you see the research Crew triggered. In this case, if you use the same terminal as before, only run `crewai run`.  

Congratulations on finishing this lab! We hope you found this new way of coding your crews exciting and fun.